In [1]:
## ODE
from scipy.integrate import odeint, solve_ivp
import numpy as np

from ipynb.fs.full.myfun_nn import *

2023-08-22 15:32:03.917554: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Tensorflow/Keras: 2.11.0
sklearn: 1.3.0


# Define the LWR models

## Lin and Log models

In [ ]:
## Traffic dynamic with LWR-model

def TD_LWR_model(t, X, N, v0, L, flag):
    """
    Lighthill-Whitham-Richards (LWR) traffic flow model in 1D
    """
    #N, v0, L, flag = args[0], args[1], args[2], args[3]
    
    # W function
    match flag:
        case "Lin":
            W = lambda z: v0*(1-1/z)
        case "Log":
            W = lambda z: v0*np.log(z)
        case _:
            return f"No match for {flag}, you can only choose between \"Lin\" and \"Log\""

    # ode sys
    d_x = np.zeros(N)
    
    for i in range(0,N-1):
        tmp = (X[i+1] - X[i])/L
        d_x[i] = W(tmp)

    d_x[N-1] = v0
        
    return d_x

## NN driven model

In [ ]:
## Traffic dynamic with ANN

def TD_ANN_model(t, X, vel):
    
    """
    Lighthill-Whitham-Richards (LWR) traffic flow model in 1D
    Transform a list as vel into a function of t,X.
    """
    
    d_x = vel
        
    return d_x

### Ode solver for the NN driven model

In [ ]:
def time_discretization(t0, tend, deltat=0.05):
    
    Nt = round((tend-t0)/deltat) + 1               # number of discretization points
                                                   # cast the value into int, us round to avoid cast problem
    tspan = np.linspace(t0, tend, int(Nt))         # timespan
    
    return tspan

In [ ]:
def odesolver_ann(x0, vel, t0, tend, deltat = 0.05):
    
    """
    odesolver_ann solves the TD_ANN_model ode system:
    * in [t0, tend] with timestep as deltat,
    * starting from x0
    * vel is the rhs passed to TD_ANN_model to create the model
    """
    
    tspan_ann = time_discretization(t0,tend,deltat)
        
    sol_ann = odeint(TD_ANN_model, x0, tspan_ann, args=(vel,), tfirst = True).T

    return tspan_ann, sol_ann

### Solve the NN-driven model in a scene

In [ ]:
def create_data_ann_scene(scn):
    
    """
    create_data_ann_scene gives the X and y for an entire scene
    
    X_scn, y_scn = create_data_ann_scene(scn)
    
    X_scn is a list of consecutive distances btw the vehicles of a scene, at each timestamps
    y_scn is a list of approximated velocities, of all the vehicles except the leader one.
    """
    
    ## Create X
    X_scn = scn['Cons Dis']
    
    ## Create y
    dX_scn = np.diff(scn['Xarr'],axis=1)
    dT_scn = np.diff(scn['Tarr'])
    velocity = dX_scn/dT_scn # velocity at the timestamps

    # we choose the first velocity discretized as (x_(i+1)-x_i)/deltaT
    y_scn = velocity[:-1] #drop the last vehicle
    
    return X_scn, y_scn

In [ ]:
def odesolver_ann_scene(nn_model, scn, epochs, batch_size, v0, deltat=0.05, verbose="auto"):
    
    """
    t_ann_list, x_ann_list, vel_ann_list = odesolver_ann_scene(nn_model, scn, epochs, batch_size, v0, deltat=0.05, verbose="auto")
    
    odesolver_ann_scene solve the ode model driven by a nn in a scene.
    """
    
    N = scn['N. vehicles']
    tstamps = scn['Tarr']
    
    vel_ann_list = []
    t_ann_list = [scn['Tarr'][0]]
    x_ann_list = [[i] for i in scn['Xarr'][:,0]]

    X_arr, y_arr = create_data_ann_scene(scn)
    
    formatter = '{0:.02f}'
    
    print("=="*50)
    print(f"We have {len(tstamps)-1} time intervals inside [{formatter.format(tstamps[0])},{formatter.format(tstamps[-1])}]\n")
    
    for i in range(0,len(tstamps)-1):

        print("--"*50)
        print(f"Time interval n.{i}: [{formatter.format(scn['Tarr'][i])}, {formatter.format(scn['Tarr'][i+1])}]\n")
        
        ## STEP 1: Neural Network
        # Create the dataset and train the nn model
        X, y = X_arr[:,i], y_arr[:,i]
        train_nn(nn_model, X, y, epochs, batch_size, verbose)
        
        # Predict the rhs of the TD_ANN_model
        y_pred = nn_model(X).numpy().flatten().tolist()
        vel_ann_list.append(np.append(y_pred,v0).tolist())

        print(f"\
        * y_true: {y}\n\
        * y_pred: {y_pred}\n")
        
        ## STEP 2: Solve the ODE sys in this time interval
        x0 = [l[-1] for l in np.vstack(x_ann_list).tolist()]  # last values computed
        t0, tend = scn['Tarr'][i], scn['Tarr'][i+1]
        vel_ann = vel_ann_list[i]
        tspan_ann, sol_ann = odesolver_ann(x0, vel_ann, t0, tend, deltat)

        ## STEP 3: Update x_ann_list, t_ann_list
        x_ann = sol_ann.tolist()
        t_ann = tspan_ann[1:] # avoid the first recording

        # add sol to the correct veh
        for j in range(0,N):
            tmp = x_ann[j][1:] # avoid the first recording
            x_ann_list[j] = np.concatenate([x_ann_list[j],tmp])
        t_ann_list = np.concatenate([t_ann_list,t_ann]).tolist()

        print("--"*50)
    
    print("=="*50)
    
    return t_ann_list, x_ann_list, vel_ann_list

In [2]:
def match_timestamps_scene(t, x, deltat = 0.05):
    
    """
    Match the computed solution to the same timestamps of the scene
    
    t_matched, x_matched = match_timestamps_scene(t, x, deltat = 0.05)
    """
    # To recover the same timestep in the data
    factor = int(0.2/deltat)
    
    t_matched = np.array(t)[::factor]
    x_matched = np.array([traj[::factor] for traj in x])
    
    return t_matched, x_matched

### Solve the NN-driven model in a sequence (more scenes $\rightarrow$ one df)

In [ ]:
def create_data_ann_seq(df):
    
    """
    
    """
    
    X_seq, y_seq = [],[]
    
    # extract input and target for each scene
    for row in df.iterrows(): #run over rows
        scn = row[1]
        X_scn, y_scn = create_data_ann_scene(scn)
        
        # store in a list
        X_seq.append(X_scn)
        y_seq.append(y_scn)

    return X_seq, y_seq

In [ ]:
def seq2scn(df):
    
    """
    
    """
    
    seq = []
    
    # extract input and target for each scene
    for row in df.iterrows(): #run over rows
        scn = row[1]
        seq.append(scn)

    return seq